In [1]:
from gymnasium import Env
from gymnasium.spaces import Box
from typing import Any, SupportsFloat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdt
import random

from grid import Microgrid
from agents import MLAgent

from stable_baselines3 import A2C

import warnings
warnings.filterwarnings("ignore")

2023-10-18 18:53:00.367695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_parquet('./data/power_price.parquet')
df = df.dropna()
days = df['time'].dt.date.unique()

# dataset = []

# for day in days:
#     part = df[df['time'].dt.date == day]
#     if len(part) == 288:
#         dataset.append({'power' : part['power'].to_numpy(),
#                         'price' : part['price'].to_numpy()})

env = Microgrid()
env.reset(df)

(array([0. , 0.2, 1. , 0. , 0. ]), {'ts_start': 15188})

In [3]:
model = A2C('MlpPolicy', env, verbose=0)
m = model.learn(total_timesteps=288000)

In [4]:
def simpleAlgo(state):
    t, batt, price, sol, hydro = state
    return np.array([1, 1 - price])


In [5]:
df = pd.read_csv('data/HOLLISTR_1_N101.csv')
df = df[['INTERVALSTARTTIME_GMT', 'MW', 'LMP_TYPE']]
df = df[df['LMP_TYPE'] == 'LMP']
df.drop(columns=['LMP_TYPE'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.columns = ['dt', 'MW']
df['dt'] = pd.to_datetime(df['dt'].str.slice(0, -6).str.replace('T', ' '))
# df['date'] = df['dt'].dt.date
# df['time'] =  df['dt'].dt.hour * 12 + df['dt'].dt.minute // 5
# df.drop(columns=['dt'], inplace=True)
df

agent = MLAgent()
agent.load_history(df.set_index('dt'))

In [6]:
episodes = 10

for ep in range(episodes):
    seed = random.randint(0, 1e9)

    state, info = env.reset(seed=seed)
    done = False
    score_rand = 0
    while not done:
        action = env.action_space.sample()
        state, reward, done, trunc, info = env.step(action)
        score_rand += reward
    

    state, info = env.reset(seed=seed)
    done = False
    score_model = 0
    while not done:
        action, state = model.predict(state)
        state, reward, done, trunc, info = env.step(action)
        score_model += reward
    
    state, info = env.reset(seed=seed)
    done = False
    score_algo = 0
    while not done:
        action = simpleAlgo(state)
        state, reward, done, trunc, info = env.step(action)
        score_algo += reward
    
    state, info = env.reset(seed=seed)
    agent.reset(info)
    done = False
    score_agent = 0
    while not done:
        action = agent.predict(state)
        state, reward, done, trunc, info = env.step(action)
        score_agent += reward
    print(f'Episode {ep+1}, score_rand {score_rand}, score_model {score_model},\
score_algo {score_algo}, score_agent {score_agent}')

Episode 1, score_rand 386.1124799341898, score_model 0,score_algo 567.79098137442, score_agent 432.389725826874


In [ ]:
seed = random.randint(0, 1e9)

state, info = env.reset(seed=seed)
done = False
score_rand = 0
actions_rand = []
while not done:
    action = env.action_space.sample()
    actions_rand.append(action)
    state, reward, done, trunc, info = env.step(action)
    score_rand += reward
actions_rand = np.array(actions_rand)

state, info = env.reset(seed=seed)
done = False
score_model = 0
actions_model = []
while not done:
    action, state = model.predict(state)
    actions_model.append(action.copy())
    state, reward, done, trunc, info = env.step(action)
    score_model += reward
actions_model = np.array(actions_model)

state, info = env.reset(seed=seed)
done = False
score_algo = 0
actions_algo = []
while not done:
    action = simpleAlgo(state)
    actions_algo.append(action)
    state, reward, done, trunc, info = env.step(action)
    score_algo += reward
actions_algo = np.array(actions_algo)
print(f'Episode {ep+1}, score_rand {score_rand}, score_model {score_model}, score_algo {score_algo}')

plt.plot(actions_model[:,0])

NameError: name 'model' is not defined